# C aceto GSM gsmodutils test suite

This notebook demonstrates C. autoethanogenum GSM tests and analysis (Norman et al., 2018)

click on cells and press 'run' to view results
requires COBRApy (https://opencobra.github.io/cobrapy/) and gsmodutils (https://github.com/sbrcnottingham/gsmodutils)

The first thing is to load up the required modules and load the project.

In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
import numpy as np
import json
import cobra
import gsmodutils
project = gsmodutils.GSMProject()

Here we view the project and run the tests with the gsmodutils command line interface.
The info command below first shows us all the models (top level gsms), designs (additional reactions or constraints to apply to models) and conditions  (growth media settings) associated with the project.

In [2]:
!command gsmodutils info

--------------------------------------------------------------------------------
Project description - Clostridium auto (Nottingham/Brookes)
Author(s): - Rupert Norman
Author email - rupert.norman@nottingham.ac.uk
Designs directory - designs
Tests directory - tests   
    
Models:
	* metaclau.json
		 metaclau
Designs:
********************************************************************************
	* flux minimisation fluxm_flux_minimisation
		 
    Minimisation of flux accross reactions as objective function
    minimize absolute sum of enzyme-catalysed network flux (Row 1 of Table 3 in manuscript)
    
********************************************************************************
	* flux minimisation h2 rest fluxm_flux_minimisation_h2_restricted
	Parent: fluxm_flux_minimisation
		
    flux minimisation with h2 restriction
    
********************************************************************************
Conditions:
		No growth conditions found
-----------------------------------

The test command runs the default tests, ensuring that all models, designs and conditions load and grow as specified.
The command then also runs the custom JSON and Python tests found in the tests folder.

This should show which aspects of models run correctly. Carefully designed tests will hilight when modelling errors occurs.
This is particularly important for propagating models.

The designs are loaded from python files in the designs folder of the project, with tests being loaded from the test folder.

Here we use verbose mode which shows all the success tests with their conditions.

In [32]:
!command gsmodutils test

------------------------- gsmodutils test results -------------------------
Running tests: 
100%|███████████████████████████████████████████| 10/10 [00:26<00:00,  2.68s/it]

Default project file tests (models, designs, conditions):
Counted 3 test assertions with 0 failures
Project file completed all tests without error
    --model_metaclau.json

    --design_fluxm_flux_minimisation

    --design_fluxm_flux_minimisation_h2_restricted

Test file test_defaults.py:
Counted 517 test assertions with 8 failures
Test file has test errors
    --test_atpase_validity
        --('metaclau.json', 'fluxm_flux_minimisation')

        --('metaclau.json', 'fluxm_flux_minimisation_h2_restricted')

        --metaclau.json


    --test_acetate

    --test_minimisation_reactions
        --('metaclau.json', 'fluxm_flux_minimisation')

        --('metaclau.json', 'fluxm_flux_minimisation_h2_restricted')


    --test_minimisation_fluxm_products
        --('metaclau.json', 'fluxm_flux_minimisation')


    --te

Alternatively we can run the tests with python an load the logs with the test log viewer.
The returned object is a dictionary showing the status of all tests and their children (assertions made in tests).

In [16]:
test_logs = project.run_tests()

100%|██████████| 9/9 [00:10<00:00,  1.12s/it]


## Performing analysis
We would like to perform more indepth analysis with our models. The following sections highlight some of the results presented in the initial C aceto genom scale model paper (Norman et al. 2018).




In [18]:
model = project.load_model()

model.summary()

IN FLUXES                   OUT FLUXES                 OBJECTIVES
--------------------------  -------------------------  ------------------
x_CARBON-MON...  10         x_CARBON-DIO...  5.04      EX_BIOMASS  0.0106
x_WATER           4.77      x_ACET           2.31
x_AMMONIA         0.0731    x_PROTON         2.28
x_Pi              0.0627    x_PPI            0.0194
x_HS              0.0018    x_BIOMASS        0.0106
x_NIACINE         0.000958  x_ADENOSYL-H...  0.000636
                            x_GLYCOL         0.000451
